In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Model Architecture

In [ ]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Flatten, Dense, Conv3D, MaxPooling3D, Dropout, LSTM, Input, Concatenate, BatchNormalization
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from keras.models import Model
from keras.regularizers import l2
import numpy as np

In [ ]:
# Input shapes
CNN_in = Input(shape=(30, 200, 400, 3))

CNN1 = Conv3D(filters=32, kernel_size=(5,5,5), activation='elu', padding='same',kernel_initializer='he_normal', name='conv2d_1')(CNN_in)
B1 = BatchNormalization(name='batchnorm_1')(CNN1)
MP1 = MaxPooling3D(pool_size=(2,2,2), name='maxpool2d_1')(B1)
D1 = Dropout(0.2)(MP1)
CNN2 = Conv3D(filters=64 ,kernel_size=(3,3,3),activation='elu',padding='same',kernel_initializer='he_normal', name='conv2d_2')(D1)
B2 = BatchNormalization(name='batchnorm_2')(CNN2)
MP2 = MaxPooling3D(pool_size=(2,2,2), name='maxpool2d_2')(B2)
D2 = Dropout(0.4)(MP2)
CNN3 = Conv3D(filters=128, kernel_size=(3,3,3),activation='elu',padding='same',kernel_initializer='he_normal', name='conv2d_3')(D2)
B3 = BatchNormalization(name='batchnorm_3')(CNN3)
MP3 = MaxPooling3D(pool_size=(2,2,2), name='maxpool2d_3')(B3)
D3 = Dropout(0.2)(MP3)
CNN4 = Conv3D(filters=128,kernel_size=(3,3,3),activation='elu',padding='same',kernel_initializer='he_normal', name='conv2d_4')(D3)
B4 = BatchNormalization(name='batchnorm_4')(CNN4)
MP4 = MaxPooling3D(pool_size=(2,2,2), name='maxpool2d_4')(B4)
D4 = Dropout(0.4)(MP4)

# Flatten
F1 = Flatten()(D4)

# Dense layers
DS1 = Dense(32, activation='relu', kernel_initializer='he_normal')(F1)
output = Dense(10, activation='softmax')(DS1)

# Create the model
model = Model(inputs = CNN_in, outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30, 200, 400, 3   0         
                             )]                                  
                                                                 
 conv2d_1 (Conv3D)           (None, 30, 200, 400, 32   12032     
                             )                                   
                                                                 
 batchnorm_1 (BatchNormaliz  (None, 30, 200, 400, 32   128       
 ation)                      )                                   
                                                                 
 maxpool2d_1 (MaxPooling3D)  (None, 15, 100, 200, 32   0         
                             )                                   
                                                                 
 dropout (Dropout)           (None, 15, 100, 200, 32   0     

## Load Npz

In [ ]:
import numpy as np

# Load the keypoints data from the npz file
data1 = np.load('/content/drive/MyDrive/lsa64/Pixel_Npzs/pixels_data_i3_j4_k3.npz')

labels = data1['labels']
pixels = data1['pixels']


print("Label:")
print(type(labels.item()))

print("pixels:")
print(pixels.shape)

Label:
<class 'str'>
pixels:
(23, 200, 400, 3)


## Padding Pixels

In [ ]:
def pad_or_truncate_pixels(pixels, target_frames=30):
    current_frames, frame_height, frame_width, _ = pixels.shape

    if current_frames < target_frames:
        # If there are fewer frames, pad the pixels with the last frame
        padding = target_frames - current_frames
        pad_frame = pixels[-1:]  # Use the last frame for padding
        pixels = np.concatenate([pixels] + [pad_frame] * padding)
    elif current_frames > target_frames:
        # If there are more frames, truncate from the back
        pixels = pixels[-target_frames:]

    return pixels

## Early Stopping

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy',min_delta=0.00005,patience=7,verbose=1,
    restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy',factor=0.5,patience=4,min_lr=1e-7,verbose=1)

callbacks = [early_stopping,lr_scheduler]

In [ ]:
pixels_folder = '/content/drive/MyDrive/lsa64/Pixel_Npzs'
batch_size = 3

In [ ]:
def load_pixel_data(label, person, selected_videos):
    pixel_data = []
    labels_data = []
    for k in selected_videos:
        filename = f'pixels_data_i{label}_j{person}_k{k}.npz'
        file_path = os.path.join(pixels_folder, filename)
        data = np.load(file_path)
        label = int(data['labels'])
        labels_data.append(label)
        pixel_data.append(pad_or_truncate_pixels(data['pixels']))  # Adjust this based on your data structure
    return np.stack(pixel_data, axis=0), labels_data

In [ ]:
from tensorflow.keras.utils import to_categorical
import numpy as np

def train_batch_generator():
    num_labels = 10
    num_persons = 10
    selected_videos = [1, 2, 3]  # Specify the selected videos (k values)
    batch_size = 3

    while True:
        for label in range(1, num_labels+1):
            for person in range(1, num_persons + 1):
                # keypoints_batch = load_keypoints_data(label, person, selected_videos)
                pixel_batch, labels_batch = load_pixel_data(label, person, selected_videos)
                num_videos = len(selected_videos)
                num_batches = num_videos // batch_size

                for batch_num in range(num_batches):
                    start = batch_num * batch_size
                    end = (batch_num + 1) * batch_size
                    labels_array = np.array(labels_batch) - 1
                    one_hot_labels = to_categorical(labels_array, num_labels)
                    yield (
                        pixel_batch[start:end]
                        # keypoints_batch[start:end]
                    ), one_hot_labels[start:end]
                    del pixel_batch, one_hot_labels


In [ ]:
def val_batch_generator():
    num_labels = 10
    num_persons = 10
    selected_videos = [4]  # Specify the selected videos (k values)
    batch_size = 1

    while True:
        for label in range(1, num_labels+1):
            for person in range(1, num_persons + 1):
                # keypoints_batch = load_keypoints_data(label, person, selected_videos)
                pixel_batch, labels_batch = load_pixel_data(label, person, selected_videos)
                num_videos = len(selected_videos)
                num_batches = num_videos // batch_size

                for batch_num in range(num_batches):
                    start = batch_num * batch_size
                    end = (batch_num + 1) * batch_size
                    labels_array = np.array(labels_batch) - 1
                    one_hot_labels = to_categorical(labels_array, num_labels)
                    yield (
                        pixel_batch[start:end]
                        # keypoints_batch[start:end]
                    ), one_hot_labels[start:end]
                    del pixel_batch, one_hot_labels

In [ ]:
import os
train_generator = train_batch_generator()
val_generator = val_batch_generator()

history = model.fit(train_generator, batch_size = 3,steps_per_epoch = 101, epochs = 20, validation_data = val_generator
                    ,validation_steps = 101, validation_batch_size=1, callbacks=callbacks)

Epoch 1/20
101/101 [==============================] - 391s 4s/step - loss: 25.8177 - accuracy: 0.4356 - val_loss: 8.5368 - val_accuracy: 0.0198 - lr: 0.0010
Epoch 2/20
101/101 [==============================] - 54s 539ms/step - loss: 2.4363 - accuracy: 0.0000e+00 - val_loss: 2.5995 - val_accuracy: 0.1287 - lr: 0.0010
Epoch 3/20
101/101 [==============================] - 55s 548ms/step - loss: 2.4566 - accuracy: 0.0132 - val_loss: 2.1264 - val_accuracy: 0.1782 - lr: 0.0010
Epoch 4/20
101/101 [==============================] - 52s 517ms/step - loss: 2.3282 - accuracy: 0.0825 - val_loss: 2.2765 - val_accuracy: 0.1683 - lr: 0.0010
Epoch 5/20
101/101 [==============================] - 56s 556ms/step - loss: 2.2885 - accuracy: 0.0099 - val_loss: 2.2589 - val_accuracy: 0.1782 - lr: 0.0010
Epoch 6/20
101/101 [==============================] - 53s 525ms/step - loss: 2.2885 - accuracy: 0.0099 - val_loss: 2.2521 - val_accuracy: 0.1683 - lr: 0.0010
Epoch 7/20
101/101 [=============================

In [ ]:
def test_batch_generator():
    num_labels = 10
    num_persons = 10
    selected_videos = [5]  # Specify the selected videos (k values)
    batch_size = 1

    while True:
        for label in range(1, num_labels + 1):
            for person in range(1, num_persons + 1):
                # keypoints_batch = load_keypoints_data(label, person, selected_videos)
                pixel_batch, labels_batch = load_pixel_data(label, person, selected_videos)
                num_videos = len(selected_videos)
                num_batches = num_videos // batch_size

                for batch_num in range(num_batches):
                    start = batch_num * batch_size
                    end = (batch_num + 1) * batch_size
                    labels_array = np.array(labels_batch) - 1
                    one_hot_labels = to_categorical(labels_array, num_labels)
                    yield (
                        pixel_batch[start:end]
                        # keypoints_batch[start:end]
                    ), one_hot_labels[start:end]
                    del pixel_batch, one_hot_labels



In [ ]:
test_generator = test_batch_generator()
test_results = model.evaluate(test_generator, steps=101)

101/101 [==============================] - 92s 917ms/step - loss: 2.1194 - accuracy: 0.1881


In [ ]:
test_results

[1.0733212232589722, 0.6393442749977112]